# Model for single mode squeezed vacuum and squeezed coherent state

Use the multihead (2-head) gates in the phase space 
to create a network that represent a squezed vaccum and a squeezed coherent state,
by starting from a Gaussian state and making a pullback

<img src="../img/singlemodesqueezing.png" widht="600"/>


<img src="../img/logo_circular.png" width="20" height="20" />@by claudio<br>
nonlinearxwaves@gmail.com<br>
@created 8 january 2021<br>
@version 15 may 2023

In [1]:
import numpy as np
from scipy.linalg import expm, sinm, cosm
from thqml import phasespace as ps
from thqml.utilities import utilities
import tensorflow as tf
#import tensorflow_addons as tfa
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

2023-05-14 11:20:15.930659: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-05-14 11:20:15.930681: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
tf_complex = tf.complex
tf_real = tf.float64
np_complex = complex
np_real = np.float64

In [3]:
tf.keras.backend.clear_session()

In [4]:
np.set_printoptions(precision=2)

Set the layer to float64

## Dimension

In [5]:
N = 4

## Symplectic matrix

In [6]:
RP, RQ, J = ps.RQRP(N)

## Index of the squeezed mode (between 0 an N/2)

In [7]:
n_squeezed=0

## Build vacuum by the Gaussian state

In [8]:
vacuum = ps.VacuumLayer(N)

2023-05-14 11:20:18.244482: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-14 11:20:18.244507: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-14 11:20:18.244529: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (x1): /proc/driver/nvidia/version does not exist
2023-05-14 11:20:18.245233: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Single mode squeezer

In [9]:
r_np=1.0;
theta_np=np.pi/2;

In [10]:
squeezer=ps.SingleModeSqueezerLayer(N, r_np=r_np, theta_np=theta_np, n_squeezed=n_squeezed)

In [11]:
Ms, _=squeezer.get_M(); Ms_np = Ms.numpy();utilities.printonscreennp(Ms_np)

+1.5+0.0i -1.2+0.0i +0.0+0.0i +0.0+0.0i 
-1.2+0.0i +1.5+0.0i +0.0+0.0i +0.0+0.0i 
+0.0+0.0i +0.0+0.0i +1.0+0.0i +0.0+0.0i 
+0.0+0.0i +0.0+0.0i +0.0+0.0i +1.0+0.0i 


## Build the model

In [12]:
xin = tf.keras.layers.Input(N)
x1, a1 = squeezer(xin)
chir, chii = vacuum(x1, a1)
single_mode_squeezed = tf.keras.Model(inputs = xin, outputs=[chir, chii])

## Evaluate the covariance matrix

In [13]:
cov_layer = ps.CovarianceLayer(N)
covariance_matrix, mean_R, _ = cov_layer(chir,chii, single_mode_squeezed)
squeezed_cov = tf.keras.Model(inputs = xin, outputs=[covariance_matrix, mean_R])

## Training points 
Here they are dummy as no training is done

In [14]:
xtrain = np.random.rand(1, N)-0.5

## Covariance and displacement

In [15]:
cov0,d0=squeezed_cov(xtrain); print(d0); tf.print(cov0)

tf.Tensor([[0. 0. 0. 0.]], shape=(1, 4), dtype=float32)
[[3.76219535 -3.62686014 -0 -0]
 [-3.62686014 3.76219535 -0 -0]
 [-0 -0 1 -0]
 [-0 -0 -0 1]]


### Eigenvalues of the covariance matrix

In [16]:
np.linalg.eig(cov0.numpy())

(array([7.39, 0.14, 1.  , 1.  ], dtype=float32),
 array([[ 0.71,  0.71,  0.  ,  0.  ],
        [-0.71,  0.71,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  1.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  1.  ]], dtype=float32))

### Uncertainty principle by the covariance matrix (g>0 and g+i J>=0, det g=1)

In [17]:
sigmaR=0.5*cov0.numpy()

In [18]:
w, v =np.linalg.eig(sigmaR)

In [19]:
print(w)

[3.69 0.07 0.5  0.5 ]


In [20]:
np.linalg.det(cov0.numpy())

0.99999934

In [21]:
print(np.linalg.eig(sigmaR+0.5*1j*J)[0])

[ 3.76e+00+2.78e-17j -4.29e-08-2.78e-17j  1.00e+00+0.00e+00j
  7.22e-33+0.00e+00j]


In [22]:
print(np.linalg.eig(cov0.numpy()+1j*J)[0])

[ 7.52e+00+5.55e-17j -8.58e-08-5.55e-17j  2.00e+00+0.00e+00j
  0.00e+00+0.00e+00j]


### Symplectic eigenvalues

In [23]:
np.matmul(J.transpose(),J)

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]], dtype=float32)

In [24]:
ws, vs =np.linalg.eig(J.transpose()*(sigmaR+0.5*1j*J))

In [25]:
print(ws)

[0.00e+00-1.88j 0.00e+00+1.88j 0.00e+00+0.5j  1.39e-17-0.5j ]


In [26]:
print(np.abs(ws))

[1.88 1.88 0.5  0.5 ]


## Count the photons and their uncertainties in the squeezed state with the HeisenbergGaussianLayer

In [27]:
photon_counter=ps.HeisenbergGaussianLayer(N) # define the layer
n_out, n2_out, Dn2_out = photon_counter(chir,chii, single_mode_squeezed);  # define the output tensor
NphotonH = tf.keras.Model(inputs = xin, outputs=[n_out,n2_out, Dn2_out]) # define the model with inputs and ouputs
no, no2, Dno2= NphotonH(xtrain);

Expected number of photons

In [28]:
tf.print(no)

[[1.38109767 0]]


The number of expected photons in the squeezed mode is sinh(r)^2 (for the other mode is zero)

In [29]:
no_th=np.sinh(r_np)**2
print(no_th)

1.3810978455418155


Mean value of expected photon number square

In [30]:
tf.print(no2)

[[8.48448753 0]]


The mean of n square is 3 no^2+2 no for the squeezed mode (for the other is zero)

In [31]:
no2_th = 3*(no_th**2)+2*no_th
print(no2_th)

8.484489467964364


Variance of photon number 

In [32]:
tf.print(Dno2)

[[6.57705688 0]]


The variance of photon number for the squeezed mode (for the other is zero)

In [33]:
Dno2_th = 2*(no_th**2)+2*no_th
print(Dno2_th)

6.57705820900412


# Model for squeezed coherent state (displaced squeezed vacuum)

## Linear displacement operator

In [34]:
dtarget = 3.0*np.ones((N,1))
displacer = ps.DisplacementLayerConstant(dtarget)

## Displacement applied to squeezed vacuum

In [35]:
xin = tf.keras.layers.Input(N)
x2, a2 = displacer(xin)
x1, a1 = squeezer(x2, a2)
chir, chii = vacuum(x1, a1)
single_mode_squeezed_1 = tf.keras.Model(inputs = xin, outputs=[chir, chii])

## Covariance matrix calculation

In [36]:
cov_layer = ps.CovarianceLayer(N)
cova, Re,_ = cov_layer(chir,chii, single_mode_squeezed_1)
squeezed_cov_1 = tf.keras.Model(inputs = xin, outputs=[cova, Re])

In [37]:
cov1, d1=squeezed_cov_1(xtrain); 

Print covariance matrix and displacement

In [38]:
print(cov1)

tf.Tensor(
[[ 3.76e+00 -3.63e+00 -0.00e+00 -0.00e+00]
 [-3.63e+00  3.76e+00 -0.00e+00 -0.00e+00]
 [ 1.91e-06  1.91e-06  1.00e+00 -0.00e+00]
 [ 1.91e-06  1.91e-06 -0.00e+00  1.00e+00]], shape=(4, 4), dtype=float32)


In [39]:
print(d1)

tf.Tensor([[3. 3. 3. 3.]], shape=(1, 4), dtype=float32)


Displacement is the same as the Displacement operator

The eigenvalues of the covariance matrix show that state is squeezed

In [40]:
np.linalg.eig(cov1)

(array([1.  , 1.  , 7.39, 0.14], dtype=float32),
 array([[ 0.00e+00,  0.00e+00,  7.07e-01,  7.07e-01],
        [ 0.00e+00,  0.00e+00, -7.07e-01,  7.07e-01],
        [ 1.00e+00,  0.00e+00,  0.00e+00, -3.12e-06],
        [ 0.00e+00,  1.00e+00,  0.00e+00, -3.12e-06]], dtype=float32))

The eigenvalues corresponds to those estabilished by the r parameter, i.e., exp(2r) and exp(-2r)

In [41]:
np.exp(2.0)

7.38905609893065

In [42]:
np.exp(-2.0)

0.1353352832366127

# Second model for squeezed coherent state (squeezed displaced vacuum)

Here we expect that squeezing affect the average value of the canonical variables

In [43]:
xin = tf.keras.layers.Input(N)
x2, a2 = squeezer(xin )
x1, a1 = displacer(x2, a2 )
chir, chii = vacuum(x1, a1)
single_mode_squeezed_2 = tf.keras.Model(inputs = xin, outputs=[chir, chii])
cov_layer = ps.CovarianceLayer(N)
cova, Re, _ = cov_layer(chir,chii, single_mode_squeezed_2)
squeezed_cov_2 = tf.keras.Model(inputs = xin, outputs=[cova, Re])

In [44]:
cov2, d2=squeezed_cov_2(xtrain); cov2_np=cov2.numpy();print(d2)

tf.Tensor([[1.1 1.1 3.  3. ]], shape=(1, 4), dtype=float32)


The displacement operator of the squeezed mode has changed!

### Theoretical value of the displacement

In [45]:
a0 = (3+1j*3)/np.sqrt(2); print(a0)

(2.1213203435596424+2.1213203435596424j)


In [46]:
a0tilde = a0*np.cosh(r_np)-np.conj(a0)*np.exp(1j*theta_np)*np.sinh(r_np); print(a0tilde)

(0.7803901425343329+0.7803901425343329j)


In [47]:
d0_th= np.real(a0tilde)*np.sqrt(2); print(d0_th)

1.1036383235143263


In [48]:
d1_th= np.imag(a0tilde)*np.sqrt(2); print(d1_th)

1.1036383235143263


Covariance matrix

In [49]:
print(cov2_np)

[[ 3.76e+00 -3.63e+00 -0.00e+00 -0.00e+00]
 [-3.63e+00  3.76e+00 -0.00e+00 -0.00e+00]
 [-2.38e-06 -2.38e-06  1.00e+00 -0.00e+00]
 [-2.38e-06 -2.38e-06 -0.00e+00  1.00e+00]]


and its eigevalues and eigenvectors

In [50]:
np.linalg.eig(cov2_np)

(array([1.  , 1.  , 7.39, 0.14], dtype=float32),
 array([[ 0.00e+00,  0.00e+00,  7.07e-01,  7.07e-01],
        [ 0.00e+00,  0.00e+00, -7.07e-01,  7.07e-01],
        [ 1.00e+00,  0.00e+00,  0.00e+00,  3.90e-06],
        [ 0.00e+00,  1.00e+00,  0.00e+00,  3.90e-06]], dtype=float32))

## Remarks

Note that the unitary eigevalues correspond to eigevectors spanning the space of non squeezed mode  (e.g. mode 1 if n_squeezed=0) 

While the eigevectors of the squeezed eigenvalues span the space of the squeezed mode (e.g. mode 0 if n_squeezed=0)

Also, while the displacement is changed the squeezing is the same as the squeeze operator

## Test the uncertainty by the covariance matrix

retrieve symplectic matrix

In [51]:
RP, RQ, J = ps.RQRP(N)

In [52]:
print(J)

[[ 0.  1.  0.  0.]
 [-1.  0.  0.  0.]
 [ 0.  0.  0.  1.]
 [ 0.  0. -1.  0.]]


In [53]:
w, v =np.linalg.eig(0.5*cov2_np+0.5*1j*J)

In [54]:
print(w)

[ 3.76e+00+4.56e-17j -1.49e-07+4.56e-17j  1.16e-16+5.45e-17j
  1.00e+00-9.02e-17j]


the matrix COV+1j*J >=0

In [55]:
w1, v1 =np.linalg.eig(0.5*J*cov2_np)

simplecticy eigenvalues

In [56]:
print(np.abs(w1))

[1.81 1.81 0.   0.  ]
